In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os

In [3]:
import pandas as pd

In [4]:
results = os.listdir('results')

In [5]:
results = [result for result in results if result.endswith('.pkl') and "2.27.2024" in result and "rfqr" in result and "std" in result and 'true_cv' in result and "CoverageTrue" in result]

In [6]:
results

['community_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl',
 'concrete_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl',
 'star_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl',
 'imdb_wiki_18_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl',
 'forest_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl',
 'bike_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl',
 'homes_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl',
 'cbc_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl',
 'bio_rfqr_experiment_isl_true_cv_std_randomizedconformalTrue_logFalse_recordCoverageTrue_02.27.2024.pkl']

In [7]:
output = pd.DataFrame(columns=['UACQR-P','UACQR-S', 'CQR','CQR-r', 'UACQR-P SE', 'UACQR-S SE', 'CQR SE','CQR-r SE'])
best_params_dict = dict()
for i,result in enumerate(results):
    dataset_name = result.split('_nn_experiment')[0]
    title = results[i].split('_rfqr')[0]
    result = pd.read_pickle('results/'+results[i]).drop(['draw'], axis=1)
    avg_result = result.mean()
    output.loc[title,['UACQR-P','UACQR-S', 'CQR','CQR-r']] = avg_result.to_numpy()
    output.loc[title,['UACQR-P SE','UACQR-S SE', 'CQR SE','CQR-r SE']] = result.sem().to_numpy()

    # best_params_dict[dataset_name] = result.loc[result.params.str.contains("'epoch_model_tracking': True")].groupby('params', sort=False).mean().iloc[:25].idxmin().to_dict()
    

In [8]:
output.sort_index()

,UACQR-P,UACQR-S,CQR,CQR-r,UACQR-P SE,UACQR-S SE,CQR SE,CQR-r SE
bike,0.904132,0.900184,0.899679,0.900184,0.001677,0.00194,0.00171,0.001524
bio,0.89965,0.899781,0.900547,0.899672,0.001408,0.002151,0.002037,0.002054
cbc,0.895833,0.896528,0.892593,0.893519,0.005937,0.005973,0.006088,0.006129
community,0.905388,0.901629,0.901253,0.899123,0.004681,0.003138,0.003193,0.003042
concrete,0.901699,0.896117,0.892718,0.895631,0.006396,0.005823,0.00619,0.005404
forest,0.901712,0.902055,0.902055,0.902104,0.003154,0.002998,0.003082,0.003151
homes,0.900624,0.900671,0.900856,0.901596,0.001648,0.001426,0.001297,0.001594
imdb_wiki_18,0.896787,0.896185,0.893373,0.893976,0.004911,0.005099,0.005248,0.005166
star,0.898152,0.897921,0.898268,0.897575,0.003235,0.004075,0.004002,0.004046


In [9]:
output.rename({'imdb_wiki_18':'imdb_wiki'}, inplace=True)

In [10]:
# output.drop(['blog','facebook_1','facebook_2','meps_19','meps_20','meps_21'],inplace=True)
output.drop(['blog','facebook_1','facebook_2', 'meps_19', 'meps_20','meps_21'], errors='ignore',inplace=True)

In [11]:
output.sort_index()

,UACQR-P,UACQR-S,CQR,CQR-r,UACQR-P SE,UACQR-S SE,CQR SE,CQR-r SE
bike,0.904132,0.900184,0.899679,0.900184,0.001677,0.00194,0.00171,0.001524
bio,0.89965,0.899781,0.900547,0.899672,0.001408,0.002151,0.002037,0.002054
cbc,0.895833,0.896528,0.892593,0.893519,0.005937,0.005973,0.006088,0.006129
community,0.905388,0.901629,0.901253,0.899123,0.004681,0.003138,0.003193,0.003042
concrete,0.901699,0.896117,0.892718,0.895631,0.006396,0.005823,0.00619,0.005404
forest,0.901712,0.902055,0.902055,0.902104,0.003154,0.002998,0.003082,0.003151
homes,0.900624,0.900671,0.900856,0.901596,0.001648,0.001426,0.001297,0.001594
imdb_wiki,0.896787,0.896185,0.893373,0.893976,0.004911,0.005099,0.005248,0.005166
star,0.898152,0.897921,0.898268,0.897575,0.003235,0.004075,0.004002,0.004046


In [12]:
def format_fig_and_se(row, method='CQR'):
    fig = row[method]
    se = row[method+' SE']
    if fig >1000:
        return f"{fig:.0f} ({se:.0f})"
    elif fig >1:
        return f"{fig:.3f} ({se:.3f})"
    else:
        return f"{fig:.3f} ({se:.3f})"

In [13]:
drop_columns = [c for c in output.columns if 'SE' in c]

In [14]:
from functools import partial

In [15]:
output['CQR'] = output.apply(partial(format_fig_and_se, method='CQR'), axis=1)
output['UACQR-S'] = output.apply(partial(format_fig_and_se, method='UACQR-S'), axis=1)
output['UACQR-P'] = output.apply(partial(format_fig_and_se, method='UACQR-P'), axis=1)
output['CQR-r'] = output.apply(partial(format_fig_and_se, method='CQR-r'), axis=1)

In [16]:
output.drop(columns=drop_columns, inplace=True)

In [17]:
# Export to LaTeX
latex_table = output.sort_index().rename(index=lambda x: f'\\texttt{{{x}}}').style.to_latex()


print('\_'.join(latex_table.split('_')))

\begin{tabular}{lllll}
 & UACQR-P & UACQR-S & CQR & CQR-r \\
\texttt{bike} & 0.904 (0.002) & 0.900 (0.002) & 0.900 (0.002) & 0.900 (0.002) \\
\texttt{bio} & 0.900 (0.001) & 0.900 (0.002) & 0.901 (0.002) & 0.900 (0.002) \\
\texttt{cbc} & 0.896 (0.006) & 0.897 (0.006) & 0.893 (0.006) & 0.894 (0.006) \\
\texttt{community} & 0.905 (0.005) & 0.902 (0.003) & 0.901 (0.003) & 0.899 (0.003) \\
\texttt{concrete} & 0.902 (0.006) & 0.896 (0.006) & 0.893 (0.006) & 0.896 (0.005) \\
\texttt{forest} & 0.902 (0.003) & 0.902 (0.003) & 0.902 (0.003) & 0.902 (0.003) \\
\texttt{homes} & 0.901 (0.002) & 0.901 (0.001) & 0.901 (0.001) & 0.902 (0.002) \\
\texttt{imdb\_wiki} & 0.897 (0.005) & 0.896 (0.005) & 0.893 (0.005) & 0.894 (0.005) \\
\texttt{star} & 0.898 (0.003) & 0.898 (0.004) & 0.898 (0.004) & 0.898 (0.004) \\
\end{tabular}

